# Census income classification with Keras

To download a copy of this notebook visit [github](https://github.com/slundberg/shap/tree/master/notebooks).

In [1]:
from sklearn.model_selection import train_test_split
from keras.layers import Input, Dense, Flatten, Concatenate, concatenate, Dropout, Lambda
from keras.models import Model
from keras.layers.embeddings import Embedding
from tqdm import tqdm
import shap

# print the JS visualization code to the notebook
shap.initjs()

Using TensorFlow backend.


ModuleNotFoundError: No module named 'shap'

## Load dataset

In [2]:
X,y = shap.datasets.adult()
X_display,y_display = shap.datasets.adult(display=True)

# normalize data (this is important for model convergence)
dtypes = list(zip(X.dtypes.index, map(str, X.dtypes)))
for k,dtype in dtypes:
    if dtype == "float32":
        X[k] -= X[k].mean()
        X[k] /= X[k].std()

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=7)

## Train Keras model

In [3]:
# build model
input_els = []
encoded_els = []
for k,dtype in dtypes:
    input_els.append(Input(shape=(1,)))
    if dtype == "int8":
        e = Flatten()(Embedding(X_train[k].max()+1, 1)(input_els[-1]))
    else:
        e = input_els[-1]
    encoded_els.append(e)
encoded_els = concatenate(encoded_els)
layer1 = Dropout(0.5)(Dense(100, activation="relu")(encoded_els))
out = Dense(1)(layer1)

# train model
regression = Model(inputs=input_els, outputs=[out])
regression.compile(optimizer="adam", loss='binary_crossentropy')
regression.fit(
    [X_train[k].values for k,t in dtypes],
    y_train,
    epochs=50,
    batch_size=512,
    shuffle=True,
    validation_data=([X_valid[k].values for k,t in dtypes], y_valid)
)

Train on 26048 samples, validate on 6513 samples
Epoch 1/50
26048/26048 [==============================] - 1s 28us/step - loss: 2.3308 - val_loss: 0.4450
Epoch 2/50
26048/26048 [==============================] - 0s 8us/step - loss: 1.5018 - val_loss: 0.5353
Epoch 3/50
26048/26048 [==============================] - 0s 9us/step - loss: 1.3662 - val_loss: 0.5634
Epoch 4/50
26048/26048 [==============================] - 0s 8us/step - loss: 1.3522 - val_loss: 0.6502
Epoch 5/50
26048/26048 [==============================] - 0s 8us/step - loss: 1.3053 - val_loss: 0.5451
Epoch 6/50
26048/26048 [==============================] - 0s 8us/step - loss: 1.2348 - val_loss: 0.5146
Epoch 7/50
26048/26048 [==============================] - 0s 8us/step - loss: 1.2083 - val_loss: 0.4880
Epoch 8/50
26048/26048 [==============================] - 0s 9us/step - loss: 1.2280 - val_loss: 0.7679
Epoch 9/50
26048/26048 [==============================] - 0s 8us/step - loss: 1.1979 - val_loss: 0.4658
Epoch 10/50
26

## Explain predictions

Here we take the Keras model trained above and explain why it makes different predictions for different individuals. SHAP expects model functions to take a 2D numpy array as input, so we define a wrapper function around the original Keras predict function.

In [4]:
def f(X):
    return regression.predict([X[:,i] for i in range(X.shape[1])]).flatten()

### Explain a single prediction

Here we use a selection of 50 samples from the dataset to represent "typical" feature values, and then use 500 perterbation samples to estimate the SHAP values for a given prediction. Note that this requires 500 * 50 evaluations of the model.

In [5]:
explainer = shap.KernelExplainer(f, X.iloc[:50,:])
shap_values = explainer.shap_values(X.iloc[299,:], nsamples=500)
shap.force_plot(explainer.expected_value, shap_values, X_display.iloc[299,:])

### Explain many predictions

Here we repeat the above explanation process for 50 individuals. Since we are using a sampling based approximation each explanation can take a couple seconds depending on your machine setup.

In [6]:
shap_values50 = explainer.shap_values(X.iloc[280:330,:], nsamples=500)

100%|██████████| 50/50 [00:53<00:00,  1.08s/it]


In [7]:
shap.force_plot(explainer.expected_value, shap_values50, X_display.iloc[280:330,:])